## Preperation: OpenAI API KEY

In [7]:
import json

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
print(os.getenv("OPENAI_API_KEY"))  # should show your key


## Agent1: Domain Detector

In [3]:
!pip install -U langchain-core langchain-openai

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [ ]:
!pip install -U langgraph

In [ ]:
!pip install langchain-openai langchain-core langgraph

### TBF-Test 0 for Agent_D: with a fixed knowledge database

In [14]:
# ---------- imports ----------
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langgraph.graph import StateGraph, END

import json, re
from typing import List, Dict, Any, Optional

# ---------- state ----------
class DomainDetectorState(dict):
    def __init__(
        self,
        columns: List[str],
        column_types: Dict[str, str],
        domains_found: List[Dict[str, Any]] = None,
        current_domain: Optional[str] = None,
        jargon_terms: List[str] = None,
        remaining_columns: List[str] = None,
        iteration: int = 0,
        messages: List[str] = None,
    ):
        super().__init__(
            columns=columns,
            column_types=column_types,
            domains_found=domains_found or [],
            current_domain=current_domain,
            jargon_terms=jargon_terms or [],
            remaining_columns=remaining_columns or list(columns),
            iteration=iteration,
            messages=messages or [],
        )

# ---------- LLM ----------
llm = OpenAI(temperature=0)

# ---------- think ----------
def think(state: DomainDetectorState) -> DomainDetectorState:
    if len(state["domains_found"]) >= 5 or not state["remaining_columns"]:
        return state

    prompt = PromptTemplate.from_template(
        """
        You are a domain expert.
        Remaining columns: {remaining_columns}
        Column types: {column_types}
        Domains already found: {domains_found}
        Identify ONE new domain (single word / short phrase).
        """
    )
    name = LLMChain(llm=llm, prompt=prompt).run(
        remaining_columns=state["remaining_columns"],
        column_types=state["column_types"],
        domains_found=[d["domain"] for d in state["domains_found"]],
    ).strip()

    new = state.copy()
    new["current_domain"] = name
    new["messages"].append(f"Think → {name}")
    return new

# ---------- helper ----------
def search_examples(domain: str, n: int = 5) -> List[str]:
    raw = (prompt | llm | StrOutputParser()).invoke({"domain": domain, "n": n})

    # ① Strict JSON path
    try:
        terms = json.loads(raw)
        if isinstance(terms, list) and len(terms) >= 3:
            return terms[:n]
    except Exception:
        pass
    
    match = re.search(r"\[(.*?)\]", raw_response, re.S)
    if match:
        rough = re.split(r"[\"',\[\]]+", match.group(1))
        cleaned = [t.strip() for t in rough if t.strip()]
        if cleaned:
            return cleaned[:n_terms]

    # 5️⃣  Final generic fallback
    return ["metric", "indicator", "analysis", "benchmark", "trend"][:n_terms]

# ---------- act ----------
def act(state: DomainDetectorState) -> DomainDetectorState:
    if not state["current_domain"]:
        new = state.copy(); new["iteration"] += 1; return new

    examples = search_examples(state["current_domain"])

    prompt = PromptTemplate.from_template(
        """
        Domain: {domain}
        Columns: {columns}
        Example jargon: {examples}
        Produce ≥5 NEW jargon terms (JSON array).
        """
    )
    jargon_response = LLMChain(llm=llm, prompt=prompt).run(
        domain=state["current_domain"],
        columns=state["columns"],
        examples=examples,
    )
    try:
        array_txt = re.search(r\"\\[.*?\\]\", jargon_response, re.S).group(0)
        jargon_terms = json.loads(array_txt)
    except Exception:
        jargon_terms = []

    new = state.copy()
    new[\"jargon_terms\"] = jargon_terms        # <- store the *parsed* list
    new[\"messages\"].append(f\"Act → {jargon_terms}\")
    new[\"iteration\"] += 1
    return new

# ---------- reflect ----------
def reflect(state: DomainDetectorState) -> DomainDetectorState:
    if not state["current_domain"] or not state["jargon_terms"]:
        new_state = state.copy()
        new_state[\"iteration\"] += 1
        return new_state

    cols_lower = {c.lower() for c in state["columns"]}
    valid = [t for t in state["jargon_terms"] if t.lower() not in cols_lower]

    new = state.copy()
    if len(valid) >= 3:
        new["domains_found"].append(
            {"domain": state["current_domain"], "jargon_terms": valid[:5]}
        )
        new["messages"].append(f"Reflect ✔ {state['current_domain']}")
        if new["remaining_columns"]:
            new["remaining_columns"].pop(0)
    else:
        new["messages"].append(f"Reflect ✖ {state['current_domain']}")

    new["current_domain"] = None
    new["jargon_terms"] = []
    new["iteration"] += 1
    return new

# ---------- stop rule ----------
def should_end(state: DomainDetectorState) -> str:
    if len(state["domains_found"]) >= 5 or not state["remaining_columns"] or state["iteration"] >= 10:
        return "end"
    return "continue"

# ---------- graph ----------
def build_graph():
    g = StateGraph(DomainDetectorState)
    g.add_node("think", think)
    g.add_node("act", act)
    g.add_node("reflect", reflect)
    g.add_edge("think", "act")
    g.add_edge("act", "reflect")
    g.add_conditional_edges("reflect", should_end, {"continue": "think", "end": END})
    g.set_entry_point("think")
    return g.compile()

# ---------- run ----------
def run_domain_detector(columns, column_types=None):
    column_types = column_types or {c: "text" for c in columns}
    state0 = DomainDetectorState(columns=columns, column_types=column_types)
    return build_graph().invoke(state0, config={"recursion_limit": 50})["domains_found"]

# ---- quick test ----
if __name__ == "__main__":
    cols = ["Quarter", "number_Customers", "Total_Transactions", "Revenue", "Profit"]
    print(json.dumps(run_domain_detector(cols), indent=2))


SyntaxError: unexpected character after line continuation character (1981263389.py, line 77)

In [15]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import json
import re
from typing import List, Dict, Any, Optional
from langgraph.graph import StateGraph, END

# Define the state schema
class DomainDetectorState(dict):
    """State for the domain detector agent."""
    def __init__(self, 
                 columns: List[str],
                 column_types: Dict[str, str],
                 domains_found: List[Dict[str, Any]] = None,
                 current_domain: Optional[str] = None,
                 jargon_terms: List[str] = None,
                 remaining_columns: List[str] = None,
                 iteration: int = 0,
                 messages: List[str] = None):
        
        self.columns = columns
        self.column_types = column_types
        self.domains_found = domains_found or []
        self.current_domain = current_domain
        self.jargon_terms = jargon_terms or []
        self.remaining_columns = remaining_columns or list(columns)
        self.iteration = iteration
        self.messages = messages or []
        
        super().__init__(
            columns=self.columns,
            column_types=self.column_types,
            domains_found=self.domains_found,
            current_domain=self.current_domain,
            jargon_terms=self.jargon_terms,
            remaining_columns=self.remaining_columns,
            iteration=self.iteration,
            messages=self.messages
        )

# Initialize LLM
llm = OpenAI(temperature=0)

# Define the "Think" node - Identify a potential domain
def think(state: DomainDetectorState) -> DomainDetectorState:
    """Identify a potential domain based on remaining columns."""
    
    # Skip if we already have 5 domains or no columns remain
    if len(state["domains_found"]) >= 5 or not state["remaining_columns"]:
        return state
    
    think_template = """
    You are a domain expert tasked with identifying knowledge domains from a set of data columns.
    
    Current columns to analyze: {remaining_columns}
    Column types: {column_types}
    
    Domains already identified: {domains_found}
    
    Based on the remaining columns, identify ONE new knowledge domain that best represents them.
    Consider business, scientific, or technical domains that would use such terminology.
    
    Return only the domain name as a single word or short phrase.
    """
    
    think_prompt = PromptTemplate(
        template=think_template,
        input_variables=["remaining_columns", "column_types", "domains_found"]
    )
    
    think_chain = LLMChain(llm=llm, prompt=think_prompt)
    
    domain = think_chain.run({
        "remaining_columns": state["remaining_columns"],
        "column_types": state["column_types"],
        "domains_found": [d["domain"] for d in state["domains_found"]]
    }).strip()
    
    new_state = state.copy()
    new_state["current_domain"] = domain
    new_state["messages"].append(f"Identified potential domain: {domain}")
    
    return new_state

# Mock function for search_examples
def search_examples(domain: str) -> List[str]:
    """Mock function to simulate fetching jargon hints for a domain."""
    # In a real implementation, this would call an external API or database
    domain_examples = {
        "Finance": ["ROI", "EBITDA", "liquidity", "amortization", "depreciation"],
        "Retail": ["SKU", "inventory turnover", "markdown", "POS", "shrinkage"],
        "E-commerce": ["conversion rate", "cart abandonment", "AOV", "CPC", "CTR"],
        "Sales": ["pipeline", "lead generation", "churn rate", "upselling", "quota"],
        "Marketing": ["CAC", "LTV", "engagement rate", "attribution", "funnel"],
        "Business Intelligence": ["KPI", "dashboard", "data warehouse", "ETL", "OLAP"]
    }
    
    # Default examples for domains not in our mock database
    default_examples = ["metric", "indicator", "analysis", "benchmark", "trend"]
    
    return domain_examples.get(domain, default_examples)

# Define the "Act" node - Get jargon terms
def act(state: DomainDetectorState) -> DomainDetectorState:
    
    """Generate jargon terms for the current domain."""

    # Skip only if there is *no* domain to work on
    if not state["current_domain"]:
        new_state = state.copy()
        new_state["iteration"] += 1
        return new_state
    
    # Get example jargon terms
    example_terms = search_examples(state["current_domain"])
    
    act_template = """
    You are a domain expert in {domain}.
    
    Your task is to generate at least 5 jargon terms that are commonly used in the {domain} domain 
    but do NOT appear verbatim in the following column names: {columns}.
    
    Here are some example jargon terms for this domain: {examples}
    
    Return a JSON array of strings containing ONLY the jargon terms.
    Example: ["term1", "term2", "term3", "term4", "term5"]
    """
    
    act_prompt = PromptTemplate(
        template=act_template,
        input_variables=["domain", "columns", "examples"]
    )
    
    act_chain = LLMChain(llm=llm, prompt=act_prompt)
    
    jargon_response = act_chain.run({
        "domain": state["current_domain"],
        "columns": state["columns"],
        "examples": example_terms
    })
    
    # Extract the JSON array from the response
    try:
        # Find anything that looks like a JSON array
        match = re.search(r'\[.*\]', jargon_response, re.DOTALL)
        if match:
            jargon_terms = json.loads(match.group(0))
        else:
            jargon_terms = []
    except Exception:
        jargon_terms = []
    
    new_state = state.copy()
    new_state["jargon_terms"] = jargon_terms
    new_state["messages"].append(f"Generated jargon terms: {jargon_terms}")
    new_state["iteration"] += 1
    return new_state

# Define the "Reflect" node - Validate jargon terms
def reflect(state: DomainDetectorState) -> DomainDetectorState:
    """Validate jargon terms and update domains found."""
    
    if not state["current_domain"] or not state["jargon_terms"]:
        return state
    
    # Convert column names to lowercase for case-insensitive comparison
    columns_lower = [col.lower() for col in state["columns"]]
    
    # Filter out jargon terms that appear in column names
    valid_terms = []
    for term in state["jargon_terms"]:
        if term.lower() not in columns_lower:
            valid_terms.append(term)
    
    # If we have at least 3 valid terms, add the domain to our list
    new_state = state.copy()
    if len(valid_terms) >= 3:
        domain_entry = {
            "domain": state["current_domain"],
            "jargon_terms": valid_terms[:5]  # Limit to 5 terms
        }
        new_state["domains_found"].append(domain_entry)
        new_state["messages"].append(f"Added domain: {state['current_domain']} with terms: {valid_terms[:5]}")
        
        # Remove columns that were used for this domain (optional)
        # This is a simplified approach - in a real implementation, you might want
        # to use the LLM to determine which columns were used for this domain
        if len(new_state["remaining_columns"]) > 0:
            new_state["remaining_columns"].pop(0)
    else:
        new_state["messages"].append(f"Rejected domain: {state['current_domain']} - not enough valid jargon terms")
    
    # Reset current domain and jargon terms
    new_state["current_domain"] = None
    new_state["jargon_terms"] = []
    new_state["iteration"] += 1
    
    return new_state

# Define the condition to end the graph
def should_end(state: DomainDetectorState) -> str:
    """Determine if the graph should end."""
    if len(state["domains_found"]) >= 5:
        return "end"
    if not state["remaining_columns"]:
        return "end"
    if state["iteration"] >= 10:  # Safety limit
        return "end"
    return "continue"

# Build the graph
def build_domain_detector_graph():
    """Build and return the domain detector graph."""
    workflow = StateGraph(DomainDetectorState)
    
    # Add nodes
    workflow.add_node("think", think)
    workflow.add_node("act", act)
    workflow.add_node("reflect", reflect)
    
    # Add edges
    workflow.add_edge("think", "act")
    workflow.add_edge("act", "reflect")
    
    # Add conditional edge
    workflow.add_conditional_edges(
        "reflect",
        should_end,
        {
            "continue": "think",
            "end": END
        }
    )
    
    # Set the entry point
    workflow.set_entry_point("think")
    
    return workflow.compile()

# Function to run the domain detector
def run_domain_detector(columns, column_types=None):
    """
    Run the domain detector on the given columns.
    
    Args:
        columns (list): List of column names
        column_types (dict, optional): Dictionary mapping column names to their data types
    
    Returns:
        list: List of domains with their jargon terms
    """
    if column_types is None:
        column_types = {col: "text" for col in columns}
    
    # Initialize the state
    initial_state = DomainDetectorState(
        columns=columns,
        column_types=column_types
    )
    
    # Build and run the graph
    graph = build_domain_detector_graph()
    final_state = graph.invoke(initial_state)
    
    return final_state["domains_found"]

# Example usage
if __name__ == "__main__":
    columns = ["Quarter", "number_Customers", "Total_Transactions", "Revenue", "Profit"]
    column_types = {
        "Quarter": "object",
        "number_Customers": "int",
        "Total_Transactions": "float",
        "Revenue": "float",
        "Profit": "float"
    }
    
    domains = run_domain_detector(columns, column_types)
    print(json.dumps(domains, indent=2))


[
  {
    "domain": "Business Analytics",
    "jargon_terms": [
      "KPI",
      "ROI",
      "Forecasting",
      "Segmentation",
      "Data Mining"
    ]
  },
  {
    "domain": "Business Finance",
    "jargon_terms": [
      "ROI",
      "EBITDA",
      "Liquidity",
      "Cash Flow",
      "Capital Expenditure"
    ]
  },
  {
    "domain": "Business Economics",
    "jargon_terms": [
      "demand_curve",
      "elasticity",
      "marginal_cost",
      "opportunity_cost",
      "market_share"
    ]
  },
  {
    "domain": "Business Accounting",
    "jargon_terms": [
      "ledger",
      "depreciation",
      "amortization",
      "accrual",
      "cash flow"
    ]
  },
  {
    "domain": "Business Management",
    "jargon_terms": [
      "KPI",
      "ROI",
      "SWOT",
      "P&L",
      "Forecast"
    ]
  }
]


#### - Visualization

In [43]:
!git filter-repo --path .env --invert-paths
!git push origin main --force


git: 'filter-repo' is not a git command. See 'git --help'.
Enumerating objects: 19, done.
Counting objects: 100% (19/19), done.
Delta compression using up to 8 threads
Compressing objects: 100% (14/14), done.
Writing objects: 100% (16/16), 14.48 KiB | 7.24 MiB/s, done.
Total 16 (delta 5), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (5/5), completed with 1 local object.
remote: error: GH013: Repository rule violations found for refs/heads/main.
remote: 
remote: - GITHUB PUSH PROTECTION
remote:   —————————————————————————————————————————
remote:     Resolve the following violations before pushing again
remote: 
remote:     - Push cannot contain secrets
remote: 
remote:     
remote:      (?) Learn how to resolve a blocked push
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-command-line#resolving-a-blocked-push
remote:     
remote:     
remote:       —— OpenAI API

In [44]:
!brew install git-filter-repo


zsh:1: command not found: brew


In [45]:
!pip install git-filter-repo  # May not work on all setups


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [46]:
!git filter-repo --path .env --invert-paths


Aborting: Refusing to destructively overwrite repo history since
this does not look like a fresh clone.
  (expected freshly packed repo)
Please operate on a fresh clone instead.  If you want to proceed
anyway, use --force.


In [47]:
!git push origin main --force


Enumerating objects: 19, done.
Counting objects: 100% (19/19), done.
Delta compression using up to 8 threads
Compressing objects: 100% (14/14), done.
Writing objects: 100% (16/16), 14.48 KiB | 7.24 MiB/s, done.
Total 16 (delta 5), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (5/5), completed with 1 local object.
remote: error: GH013: Repository rule violations found for refs/heads/main.
remote: 
remote: - GITHUB PUSH PROTECTION
remote:   —————————————————————————————————————————
remote:     Resolve the following violations before pushing again
remote: 
remote:     - Push cannot contain secrets
remote: 
remote:     
remote:      (?) Learn how to resolve a blocked push
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-command-line#resolving-a-blocked-push
remote:     
remote:     
remote:       —— OpenAI API Key ————————————————————————————————————
remote:        lo

In [48]:
!git filter-repo --path .env --invert-paths --force


NOTICE: Removing 'origin' remote; see 'Why is my origin removed?'
        in the manual if you want to push back there.
        (was https://github.com/77luvC/D2D_Data2Dashboard.git)
Parsed 10 commits
New history written in 0.06 seconds; now repacking/cleaning...
Repacking your repo and cleaning out old unneeded objects
HEAD is now at 22fed46 agentize domain detector
Enumerating objects: 32, done.
Counting objects: 100% (32/32), done.
Delta compression using up to 8 threads
Compressing objects: 100% (29/29), done.
Writing objects: 100% (32/32), done.
Total 32 (delta 10), reused 1 (delta 0), pack-reused 0
Completely finished after 0.14 seconds.


In [49]:
!git push origin main --force


fatal: 'origin' does not appear to be a git repository
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [50]:
!git remote add origin https://github.com/77luvC/D2D_Data2Dashboard.git


In [51]:
!git push origin main --force


Enumerating objects: 32, done.
Counting objects: 100% (32/32), done.
Delta compression using up to 8 threads
Compressing objects: 100% (19/19), done.
Writing objects: 100% (32/32), 39.08 KiB | 39.08 MiB/s, done.
Total 32 (delta 10), reused 32 (delta 10), pack-reused 0
remote: Resolving deltas: 100% (10/10), done.
To https://github.com/77luvC/D2D_Data2Dashboard.git
 + e96d856...22fed46 main -> main (forced update)


### Test 1 for Agent_D: Dynamic LLM-powered jargon fetcher

In [17]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import json
import re
from typing import List, Dict, Any, Optional
from langgraph.graph import StateGraph, END

# Define the state schema
class DomainDetectorState(dict):
    """State for the domain detector agent."""
    def __init__(self, 
                 columns: List[str],
                 column_types: Dict[str, str],
                 domains_found: List[Dict[str, Any]] = None,
                 current_domain: Optional[str] = None,
                 jargon_terms: List[str] = None,
                 remaining_columns: List[str] = None,
                 iteration: int = 0,
                 messages: List[str] = None):
        
        self.columns = columns
        self.column_types = column_types
        self.domains_found = domains_found or []
        self.current_domain = current_domain
        self.jargon_terms = jargon_terms or []
        self.remaining_columns = remaining_columns or list(columns)
        self.iteration = iteration
        self.messages = messages or []
        
        super().__init__(
            columns=self.columns,
            column_types=self.column_types,
            domains_found=self.domains_found,
            current_domain=self.current_domain,
            jargon_terms=self.jargon_terms,
            remaining_columns=self.remaining_columns,
            iteration=self.iteration,
            messages=self.messages
        )

# Initialize LLM
llm = OpenAI(temperature=0)

# Define the "Think" node - Identify a potential domain
def think(state: DomainDetectorState) -> DomainDetectorState:
    """Identify a potential domain based on remaining columns."""
    
    # Skip if we already have 5 domains or no columns remain
    if len(state["domains_found"]) >= 5 or not state["remaining_columns"]:
        return state
    
    think_template = """
    You are a domain expert tasked with identifying knowledge domains from a set of data columns.
    
    Current columns to analyze: {remaining_columns}
    Column types: {column_types}
    
    Domains already identified: {domains_found}
    
    Based on the remaining columns, identify ONE new knowledge domain that best represents them.
    Consider business, scientific, or technical domains that would use such terminology.
    
    Return only the domain name as a single word or short phrase.
    """
    
    think_prompt = PromptTemplate(
        template=think_template,
        input_variables=["remaining_columns", "column_types", "domains_found"]
    )
    
    think_chain = LLMChain(llm=llm, prompt=think_prompt)
    
    domain = think_chain.run({
        "remaining_columns": state["remaining_columns"],
        "column_types": state["column_types"],
        "domains_found": [d["domain"] for d in state["domains_found"]]
    }).strip()
    
    new_state = state.copy()
    new_state["current_domain"] = domain
    new_state["messages"].append(f"Identified potential domain: {domain}")
    
    return new_state

# -------------------------------------------------------------------
# Dynamic LLM-powered jargon fetcher
# -------------------------------------------------------------------
def search_examples(domain: str, n_terms: int = 5, llm_model=llm) -> List[str]:
    """
    Query the LLM for ~n_terms canonical jargon / acronyms that typify `domain`.
    Falls back to a small generic list if the model response is unusable.
    """

    # 1️⃣  Compose the prompt
    prompt = PromptTemplate.from_template(
        """
        You are a senior specialist in {domain}.
        List {n} widely-used jargon terms or acronyms **unique** to this field.
        Return ONLY a valid JSON array of quoted strings, nothing else.
        Example → ["term1","term2",...]
        """
    )

    # 2️⃣  Run the model
    raw_response: str = (
        prompt
        | llm_model
        | StrOutputParser()         # gives us the raw string
    ).invoke({"domain": domain, "n": n_terms})

    # 3️⃣  Try strict JSON parsing first
    try:
        terms = json.loads(raw_response)
        if isinstance(terms, list) and len(terms) >= 3:
            return terms[:n_terms]
    except Exception:
        pass

    # 4️⃣  Regex rescue for “almost JSON”
    match = re.search(r"\[(.*?)\]", raw_response, re.S)
    if match:
        rough = re.split(r"[\"',\[\]]+", match.group(1))
        cleaned = [t.strip() for t in rough if t.strip()]
        if cleaned:
            return cleaned[:n_terms]

    # 5️⃣  Final generic fallback
    return ["metric", "indicator", "analysis", "benchmark", "trend"][:n_terms]


# Define the "Act" node - Get jargon terms
def act(state: DomainDetectorState) -> DomainDetectorState:
    
    """Generate jargon terms for the current domain."""

    # Skip only if there is *no* domain to work on
    if not state["current_domain"]:
        new_state = state.copy()
        new_state["iteration"] += 1
        return new_state
    
    # Get example jargon terms
    example_terms = search_examples(state["current_domain"])
    
    act_template = """
    You are a domain expert in {domain}.
    
    Your task is to generate at least 5 jargon terms that are commonly used in the {domain} domain 
    but do NOT appear verbatim in the following column names: {columns}.
    
    Here are some example jargon terms for this domain: {examples}
    
    Return a JSON array of strings containing ONLY the jargon terms.
    Example: ["term1", "term2", "term3", "term4", "term5"]
    """
    
    act_prompt = PromptTemplate(
        template=act_template,
        input_variables=["domain", "columns", "examples"]
    )
    
    act_chain = LLMChain(llm=llm, prompt=act_prompt)
    
    jargon_response = act_chain.run({
        "domain": state["current_domain"],
        "columns": state["columns"],
        "examples": example_terms
    })
    
    # Extract the JSON array from the response
    try:
        # Find anything that looks like a JSON array
        match = re.search(r'\[.*\]', jargon_response, re.DOTALL)
        if match:
            jargon_terms = json.loads(match.group(0))
        else:
            jargon_terms = []
    except Exception:
        jargon_terms = []
    
    new_state = state.copy()
    new_state["jargon_terms"] = jargon_terms
    new_state["messages"].append(f"Generated jargon terms: {jargon_terms}")
    new_state["iteration"] += 1
    return new_state

# Define the "Reflect" node - Validate jargon terms
def reflect(state: DomainDetectorState) -> DomainDetectorState:
    """Validate jargon terms and update domains found."""
    
    if not state["current_domain"] or not state["jargon_terms"]:
        return state
    
    # Convert column names to lowercase for case-insensitive comparison
    columns_lower = [col.lower() for col in state["columns"]]
    
    # Filter out jargon terms that appear in column names
    valid_terms = []
    for term in state["jargon_terms"]:
        if term.lower() not in columns_lower:
            valid_terms.append(term)
    
    # If we have at least 3 valid terms, add the domain to our list
    new_state = state.copy()
    if len(valid_terms) >= 3:
        domain_entry = {
            "domain": state["current_domain"],
            "jargon_terms": valid_terms[:5]  # Limit to 5 terms
        }
        new_state["domains_found"].append(domain_entry)
        new_state["messages"].append(f"Added domain: {state['current_domain']} with terms: {valid_terms[:5]}")
        
        # Remove columns that were used for this domain (optional)
        # This is a simplified approach - in a real implementation, you might want
        # to use the LLM to determine which columns were used for this domain
        if len(new_state["remaining_columns"]) > 0:
            new_state["remaining_columns"].pop(0)
    else:
        new_state["messages"].append(f"Rejected domain: {state['current_domain']} - not enough valid jargon terms")
    
    # Reset current domain and jargon terms
    new_state["current_domain"] = None
    new_state["jargon_terms"] = []
    new_state["iteration"] += 1
    
    return new_state

# Define the condition to end the graph
def should_end(state: DomainDetectorState) -> str:
    """Determine if the graph should end."""
    if len(state["domains_found"]) >= 5:
        return "end"
    if not state["remaining_columns"]:
        return "end"
    if state["iteration"] >= 10:  # Safety limit
        return "end"
    return "continue"

# Build the graph
def build_domain_detector_graph():
    """Build and return the domain detector graph."""
    workflow = StateGraph(DomainDetectorState)
    
    # Add nodes
    workflow.add_node("think", think)
    workflow.add_node("act", act)
    workflow.add_node("reflect", reflect)
    
    # Add edges
    workflow.add_edge("think", "act")
    workflow.add_edge("act", "reflect")
    
    # Add conditional edge
    workflow.add_conditional_edges(
        "reflect",
        should_end,
        {
            "continue": "think",
            "end": END
        }
    )
    
    # Set the entry point
    workflow.set_entry_point("think")
    
    return workflow.compile()

# Function to run the domain detector
def run_domain_detector_test1(columns, column_types=None):
    """
    Run the domain detector on the given columns.
    
    Args:
        columns (list): List of column names
        column_types (dict, optional): Dictionary mapping column names to their data types
    
    Returns:
        list: List of domains with their jargon terms
    """
    if column_types is None:
        column_types = {col: "text" for col in columns}
    
    # Initialize the state
    initial_state = DomainDetectorState(
        columns=columns,
        column_types=column_types
    )
    
    # Build and run the graph
    graph = build_domain_detector_graph()
    final_state = graph.invoke(initial_state)
    
    return final_state["domains_found"]

# Example usage
if __name__ == "__main__":
    columns = ["Quarter", "number_Customers", "Total_Transactions", "Revenue", "Profit"]
    column_types = {
        "Quarter": "object",
        "number_Customers": "int",
        "Total_Transactions": "float",
        "Revenue": "float",
        "Profit": "float"
    }
    
    domains = run_domain_detector_test1(columns, column_types)
    print(json.dumps(domains, indent=2))


[
  {
    "domain": "Business Analytics",
    "jargon_terms": [
      "Data Mining",
      "Predictive Modeling",
      "Data Visualization",
      "Descriptive Analytics",
      "Prescriptive Analytics"
    ]
  },
  {
    "domain": "Business Finance",
    "jargon_terms": [
      "EBITDA",
      "ROI",
      "DCF",
      "IRR",
      "NPV"
    ]
  },
  {
    "domain": "Business Economics",
    "jargon_terms": [
      "Demand Curve",
      "Opportunity Cost",
      "Elasticity",
      "Monopoly",
      "Oligopoly"
    ]
  },
  {
    "domain": "Business Accounting",
    "jargon_terms": [
      "Balance Sheet",
      "Cash Flow Statement",
      "Accrual Basis",
      "Depreciation",
      "Inventory Valuation"
    ]
  },
  {
    "domain": "Business Management",
    "jargon_terms": [
      "KPI",
      "ROI",
      "SWOT",
      "B2B",
      "B2C"
    ]
  }
]


### 1. Cleaned Domain-Concept Pairs

In [57]:
from collections import Counter

# keep the first occurrence of any jargon term;
# drop later duplicates that show up in other domains.

def get_first_unique_pairs(domains_found):
    """
    Keep the first‑seen (domain, jargon) pair for every jargon_term.
    Later occurrences of the same jargon_term in other domains are skipped.
    """
    seen = set()
    unique_pairs = []

    for entry in domains_found:          # keep original order
        dom = entry["domain"]
        for term in entry["jargon_terms"]:
            if term not in seen:         # only accept the first time we see it
                unique_pairs.append({"domain": dom, "jargon_terms": term})
                seen.add(term)

    return unique_pairs

cleaned = get_first_unique_pairs(domains)
import json, pprint; pprint.pp(cleaned, width=60)



[{'domain': 'Business Analytics',
  'jargon_terms': 'Data Mining'},
 {'domain': 'Business Analytics',
  'jargon_terms': 'Predictive Modeling'},
 {'domain': 'Business Analytics',
  'jargon_terms': 'Data Visualization'},
 {'domain': 'Business Analytics',
  'jargon_terms': 'Descriptive Analytics'},
 {'domain': 'Business Analytics',
  'jargon_terms': 'Prescriptive Analytics'},
 {'domain': 'Business Finance', 'jargon_terms': 'EBITDA'},
 {'domain': 'Business Finance', 'jargon_terms': 'ROI'},
 {'domain': 'Business Finance', 'jargon_terms': 'DCF'},
 {'domain': 'Business Finance', 'jargon_terms': 'IRR'},
 {'domain': 'Business Finance', 'jargon_terms': 'NPV'},
 {'domain': 'Business Economics',
  'jargon_terms': 'Demand Curve'},
 {'domain': 'Business Economics',
  'jargon_terms': 'Opportunity Cost'},
 {'domain': 'Business Economics',
  'jargon_terms': 'Elasticity'},
 {'domain': 'Business Economics',
  'jargon_terms': 'Monopoly'},
 {'domain': 'Business Economics',
  'jargon_terms': 'Oligopoly'},
 

### 2. Filter by yes/no

In [66]:
# ------------ 1. imports & chat object ------------
from langchain_openai import ChatOpenAI
from typing import List, Dict, Union

# feel free to tweak these
MODEL_NAME   = "gpt-4o"   # or "gpt-4o", "gpt-3.5-turbo"
TEMPERATURE  = 0
MAX_TOKENS   = 3               # we only need YES / NO
API_KEY_ENV  = "OPENAI_API_KEY"

# singleton chat object (re‑used for every call)
chat = ChatOpenAI(
    model_name   = MODEL_NAME,
    temperature  = TEMPERATURE,
    max_tokens   = MAX_TOKENS,
    openai_api_key = None,     # uses env var by default
)


# ------------ 2. prompt builder ------------
def _build_prompt(jargon: str,
                  domain: str,
                  columns: List[str],
                  column_types: Dict[str, str]) -> str:
    """
    Few‑shot prompt.  The model must output TWO flags separated by a comma:
      * first flag  – YES/NO: term is relevant to the data columns
      * second flag – YES/NO: term generally has a well‑defined numeric formula
    """
    few_shots = """
### Example 1
Data columns: ["Date", "Open", "High", "Low", "Close"]
Column types: {"Date":"object","Open":"float","High":"float","Low":"float","Close":"float"}
Jargon term: "EBITDA"
Answer: YES, YES

### Example 2
Data columns: ["Epoch", "Loss", "Accuracy"]
Column types: {"Epoch":"int","Loss":"float","Accuracy":"float"}
Jargon term: "Predictive Modeling"
Answer: NO, NO

### Example 3
Data columns: ["Customer_ID", "Invoice_Amount", "Payment_Date"]
Column types: {"Customer_ID":"int","Invoice_Amount":"float","Payment_Date":"object"}
Jargon term: "ROI"
Answer: YES, YES
""".strip()

    task = f"""
### Task
Data columns: {json.dumps(columns)}
Column types: {json.dumps(column_types)}
Jargon term: "{jargon}"
Answer (format YES/NO,YES/NO — no extra text):""".rstrip()

    return f"{few_shots}\n{task}"


# ------------ 3. evaluator ------------
def lc_relevance(pairs: List[Dict[str, str]],
                 columns: List[str],
                 column_types: Dict[str, str]
                ) -> List[Dict[str, Union[str, bool]]]:
    """
    Returns a list with two Boolean flags for each jargon term:
      - relevant    : related to the dataframe columns
      - has_formula : likely has a numeric / algebraic formula
    """
    results = []
    for p in pairs:
        term   = p.get("jargon_terms") or p.get("concept")
        domain = p["domain"]

        prompt = _build_prompt(term, domain, columns, column_types)
        reply  = chat.invoke(prompt).content.strip().upper()

        # Accept variants such as "YES , NO"
        parts = [x.strip() for x in reply.split(",")]
        rel_flag   = parts[0].startswith("Y") if parts else False
        formula_fl = parts[1].startswith("Y") if len(parts) > 1 else False

        results.append({
            "domain"      : domain,
            "jargon_term" : term,
            "relevant"    : rel_flag,
            "has_formula" : formula_fl,
        })
    return results


In [67]:
evaluated = lc_relevance(cleaned, columns, column_types)

import pandas as pd
pd.DataFrame(evaluated)


,domain,jargon_term,relevant,has_formula
0,Business Analytics,Data Mining,True,False
1,Business Analytics,Predictive Modeling,True,False
2,Business Analytics,Data Visualization,False,False
3,Business Analytics,Descriptive Analytics,True,True
4,Business Analytics,Prescriptive Analytics,True,True
5,Business Finance,EBITDA,True,True
6,Business Finance,ROI,True,True
7,Business Finance,DCF,True,True
8,Business Finance,IRR,True,True
9,Business Finance,NPV,True,True


### 3. Prepare for Formula Retriever

In [68]:
from typing import List, Dict

def to_formula_targets(df: pd.DataFrame) -> List[Dict[str, str]]:
    """
    Keep only rows where   relevant == True   AND   has_formula == True,
    and return a list of plain dicts:  { "domain": ..., "jargon_term": ... }.
    """
    mask   = (df["relevant"]) & (df["has_formula"])
    subset = df.loc[mask, ["domain", "jargon_term"]]
    return subset.to_dict(orient="records")

# ---- example usage ---------------------------------------------
df         = pd.DataFrame(evaluated)     # «evaluated» is the list from step 3
targets    = to_formula_targets(df)
print(json.dumps(targets, indent=2))

[
  {
    "domain": "Business Analytics",
    "jargon_term": "Descriptive Analytics"
  },
  {
    "domain": "Business Analytics",
    "jargon_term": "Prescriptive Analytics"
  },
  {
    "domain": "Business Finance",
    "jargon_term": "EBITDA"
  },
  {
    "domain": "Business Finance",
    "jargon_term": "ROI"
  },
  {
    "domain": "Business Finance",
    "jargon_term": "DCF"
  },
  {
    "domain": "Business Finance",
    "jargon_term": "IRR"
  },
  {
    "domain": "Business Finance",
    "jargon_term": "NPV"
  },
  {
    "domain": "Business Economics",
    "jargon_term": "Opportunity Cost"
  },
  {
    "domain": "Business Economics",
    "jargon_term": "Elasticity"
  },
  {
    "domain": "Business Accounting",
    "jargon_term": "Cash Flow Statement"
  },
  {
    "domain": "Business Accounting",
    "jargon_term": "Accrual Basis"
  },
  {
    "domain": "Business Accounting",
    "jargon_term": "Depreciation"
  },
  {
    "domain": "Business Management",
    "jargon_term": "KPI"
  },

## Agent2: Formula Retriever

In [5]:
!pip install wikipedia

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [6]:
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
import json

# Initialize Wikipedia tool
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Function to search domain-specific formulas from Wikipedia
def search_domain_formulas(domain, concept):
    """Search Wikipedia for formulas related to a specific domain and concept."""
    try:
        results = wikipedia.run(f"{domain} {concept} formula")
        return results
    except Exception as e:
        return f"Error searching Wikipedia: {str(e)}"

# Initialize LLM for formula contextualization
llm = OpenAI(temperature=0)

# Create a prompt template for converting formulas to textual descriptions
formula_contextualized_prompt = PromptTemplate(
    input_variables=["formula", "domain", "concept"],
    template="""
    Convert the following mathematical formula into a clear textual explanation:
    
    Domain: {domain}
    Concept: {concept}
    Formula: {formula}
    
    Provide a step-by-step explanation of what this formula means in plain language.
    Explain each variable and how they relate to each other.
    """
)

# Create a prompt template for extracting variables from formulas
formula_variables_prompt = PromptTemplate(
    input_variables=["formula"],
    template="""
    Extract all variables from the following formula and return them in JSON format:
   
    Formula: {formula}
    
    For each variable, provide:
    1. The variable name/symbol
    2. What it represents in this context
    3. Its typical unit of measurement (if applicable)
    
    Return as a valid JSON object with this structure:
    {{
        "variables": [
            {{"name": "variable_symbol", "represents": "description", "unit": "unit_of_measurement"}}
        ]
    }}
    """
)

# Create a chain for formula contextualized
formula_contextualized_chain = LLMChain(llm=llm, prompt=formula_contextualized_prompt)

# Create a chain for extracting variables
formula_variables_chain = LLMChain(llm=llm, prompt=formula_variables_prompt)



# -------------------------------- Example usage --------------------------- #

domain_example = "Finance"
concept_example = "Return on Investment"

# Retrieve formula from Wikipedia
formula_text = search_domain_formulas(domain_example, concept_example)
print(f"Retrieved formula information for {concept_example} in {domain_example}:")
print(formula_text[:200] + "..." if len(formula_text) > 200 else formula_text)

# Convert formula to textual explanation
contextualized_formula = formula_contextualized_chain.run(
    formula=formula_text, 
    domain=domain_example, 
    concept=concept_example
)
print("\ncontextualized formula:")
print(contextualized_formula)



/var/folders/p4/f1848b4x3815qpp5kmf2j66w0000gn/T/ipykernel_1798/2626271568.py:21: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)
/var/folders/p4/f1848b4x3815qpp5kmf2j66w0000gn/T/ipykernel_1798/2626271568.py:61: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  formula_contextualized_chain = LLMChain(llm=llm, prompt=formula_contextualized_prompt)


Retrieved formula information for Return on Investment in Finance:
Page: Return on investment
Summary: Return on investment (ROI) or return on costs (ROC) is the ratio between net income (over a period) and investment (costs resulting from an investment of some resou...


/var/folders/p4/f1848b4x3815qpp5kmf2j66w0000gn/T/ipykernel_1798/2626271568.py:79: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  contextualized_formula = formula_contextualized_chain.run(



contextualized formula:

The return on investment (ROI) is a measure of how much profit or gain an investment has generated compared to the cost of the investment. It is calculated by dividing the net income (total income minus expenses) by the initial investment cost. This ratio is expressed as a percentage and is used to evaluate the efficiency of an investment or to compare different investments.

To calculate ROI, you need to know the net income and the initial investment cost. Net income is the total amount of money earned from the investment, while the initial investment cost is the amount of money that was initially put into the investment.

For example, if you invest $100 and earn a net income of $20, your ROI would be 20%. This means that for every dollar you invested, you earned an additional 20 cents in profit.

Another related concept is the rate of return, which is a way to compare returns over different time periods. This is useful because it allows for a fair comparison


Extracted variables:
Could not parse variables as JSON. Raw output:

{
    "variables": [
        {"name": "ROI", "represents": "Return on investment", "unit": "ratio"},
        {"name": "ROC", "represents": "Return on costs", "unit": "ratio"},
        {"name": "net_income", "represents": "Net income", "unit": "currency"},
        {"name": "investment", "represents": "Costs resulting from an investment", "unit": "currency"},
        {"name": "performance_measure", "represents": "Efficiency of an investment", "unit": "ratio"},
        {"name": "economics", "represents": "Profits to capital invested", "unit": "ratio"},
        {"name": "rate_of_return", "represents": "Profit on an investment", "unit": "percentage"},
        {"name": "absolute_return", "represents": "Change in value of the investment", "unit": "currency"},
        {"name": "holding_period_return", "represents": "Return over a period of time", "unit": "percentage"},
        {"name": "annualized_return", "represents": "Rat